Geonames Data

- file: https://download.geonames.org/export/dump/cities5000.zip
- all cities with a population > 5000
- data definition: https://download.geonames.org/export/dump/readme.txt

In [1]:
import pandas as pd

In [2]:
cols = [
    "geonameid",
    "name",
    "asciiname",
    "alternatenames",
    "latitude",
    "longitude",
    "feature class",
    "feature code",
    "country code",
    "cc2",
    "admin1 code",
    "admin2 code",
    "admin3 code",
    "admin4 code",
    "population",
    "elevation",
    "dem",
    "timezone",
    "modification",
]

In [3]:
df = pd.read_csv(
    "https://download.geonames.org/export/dump/cities5000.zip",
    sep="\t",
    names=cols,
    na_values=[""],
    keep_default_na=False,
)
len(df)

58648

filter data source with the following criteria:

- first-order administrative division with a population > 10000
- second/third-order administrative division with a population > 500,000
- capital cities

where feature codes are:

- PPLA - first-order administrative division
- PPLA2 - second-order administrative division
- PPLA3 - third-order administrative division
- PPLC - Capital of a country, region
- PPLCD - Capital of a dependency or special area
- PPLCH - Historical Capital

refer https://download.geonames.org/export/dump/featureCodes_en.txt for details

In [4]:
criteria = (
    ((df["feature code"] == "PPLA") & (df["population"] > 10000))
    | ((df["feature code"].isin(["PPLA2", "PPLA3"])) & (df["population"] > 500_000))
    | ((df["feature code"] == "PPL") & (df["population"] > 1_000_000))
    | (df["feature code"].isin(["PPLC", "PPLCD", "PPLCH"]))
)

- cities ordered by name with the following columns:
  - asciiname
  - population
  - timezone
  - country code
  - latitude
  - longitude
- rename the columns to the following:
  - name
  - pop
  - timezone
  - country
  - lat
  - lon

In [5]:
# unique admin1 codes
df = df[criteria][["asciiname", "population", "timezone", "country code", "latitude", "longitude"]]
df.columns = ["name", "pop", "timezone", "country", "lat", "lon"]

extra cleanup 

- remove starting single quote in the name column
- change lat and lon to 6 significant digits
- case insensitive sorting by name and population
- drop duplicates

In [6]:
df["name"] = df["name"].str.lstrip("'")
df["lat"] = df["lat"].apply(lambda x: float('{:.6g}'.format(x)))
df["lon"] = df["lon"].apply(lambda x: float('{:.6g}'.format(x)))

In [7]:
cities = df.sort_values(
    by=["name", "pop"],
    key=lambda x: x.str.lower() if x.name == "name" else x,
    ascending=[True, False],
)
cities = cities.drop_duplicates(subset=["name", "country"], keep="first")
len(cities)

3222

- concatenate Chinese cities from the following csv
- export to gzipped csv

In [8]:
from io import BytesIO

In [9]:
csv = """name,pop,timezone,country,lat,lon
台北,7871900,Asia/Taipei,TW,25.0478,121.532
新北,543342,Asia/Taipei,TW,25.0143,121.467
基隆,397515,Asia/Taipei,TW,25.1309,121.741
桃園,402014,Asia/Taipei,TW,24.9937,121.297
新竹,404109,Asia/Taipei,TW,24.8036,120.969
苗栗,533219,Asia/Taipei,TW,24.5141,120.317
台中,1040725,Asia/Taipei,TW,24.1469,120.684
彰化,124725,Asia/Taipei,TW,23.9567,120.576
雲林,104723,Asia/Taipei,TW,23.7094,120.543
嘉義,262553,Asia/Taipei,TW,23.5769,120.317
台南,771235,Asia/Taipei,TW,22.9908,120.213
高雄,1519711,Asia/Taipei,TW,22.6163,120.313
屏東,31288,Asia/Taipei,TW,22.0042,120.744
南投,105682,Asia/Taipei,TW,23.9157,120.664
宜蘭,94188,Asia/Taipei,TW,24.757,121.753
花蓮,350468,Asia/Taipei,TW,23.9769,121.604
台東,211681,Asia/Taipei,TW,22.7217,120.51
澎湖,56435,Asia/Taipei,TW,23.5654,119.586
金門,37507,Asia/Taipei,TW,24.4341,118.317
馬祖,8000,Asia/Taipei,TW,26.1457,119.913
香港,7491609,Asia/Hong_Kong,HK,22.2783,114.174
澳門,649335,Asia/Macau,MO,22.2005,113.546
北京,18960744,Asia/Shanghai,CN,39.9075,116.397
上海,22315474,Asia/Shanghai,CN,31.2222,121.458
廣州,16096724,Asia/Shanghai,CN,23.1166,113.25
深圳,17494398,Asia/Shanghai,CN,22.5455,114.068
"""

chinese_cities = pd.read_csv(BytesIO(csv.encode()))
pd.concat([chinese_cities, cities]).to_csv("natal/data/cities.csv.gz", index=False, compression="gzip")